In [4]:
import json
import os
import pandas as pd
import requests
import time

In [2]:
api_points = pd.read_csv('API_query_points.csv')[['Y','X','id']]
api_points

,Y,X,id
0,49.002394,-122.569130,3
1,49.002394,-121.895394,4
2,49.002394,-121.221657,5
3,49.002394,-119.874184,7
4,49.002394,-119.200448,8
...,...,...,...
77,45.811966,-121.221657,89
78,45.811966,-120.547921,90
79,45.811966,-119.874184,91
80,45.811966,-119.200448,92


In [3]:
api_key = ''
radius_m = '50000'
place_type = 'gas_station'
for i in range(0,len(api_points)):
    print(i)
    lat = api_points['Y'][i]
    lon = api_points['X'][i]
    point_id = api_points['id'][i]
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius={radius_m}&type={place_type}&key={api_key}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    response_json = json.loads(response.text)
    j=0

    # Dump whole response to json file
    with open(f"google_point_{point_id}_page_{j}.json", 'w') as outfile:
        json.dump(response_json, outfile)

    # Write minimal variables to csv file
    with open(f"google_point_{point_id}_page_{j}.json") as json_file:
        data = json.load(json_file)
        all_rows = []
        for result in data['results']:
            current_row = []
            current_row.append(result['place_id'])
            current_row.append(result['geometry']['location']['lat'])
            current_row.append(result['geometry']['location']['lng'])
            current_row.append(result['business_status'])
            current_row.append(result['name'])
            current_row.append(result['vicinity'])
            all_rows.append(current_row)
    api_point_df = pd.DataFrame(all_rows, columns=['place_id','lat','lon','business_status','name','vicinity'])
    api_point_df.to_csv(f"google_point_{point_id}_page_{j}.csv")
    
    # Google returns 20 results per page, up to max 60 results total
    while 'next_page_token' in response_json.keys():
        time.sleep(2)
        j+=1
        url += ("&pagetoken=" + response_json['next_page_token'])
        payload={}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        response_json = json.loads(response.text)
        
        # Check if last page (starts to repeat)
        if api_point_df.iloc[0,0] == response_json['results'][0]['place_id']:
            break

        # Dump whole response to json file
        with open(f"google_point_{point_id}_page_{j}.json", 'w') as outfile:
            json.dump(response_json, outfile)

        # Write minimal variables to csv file
        with open(f"google_point_{point_id}_page_{j}.json") as json_file:
            data = json.load(json_file)
            all_rows = []
            for result in data['results']:
                current_row = []
                current_row.append(result['place_id'])
                current_row.append(result['geometry']['location']['lat'])
                current_row.append(result['geometry']['location']['lng'])
                current_row.append(result['business_status'])
                current_row.append(result['name'])
                current_row.append(result['vicinity'])
                all_rows.append(current_row)
        api_point_df = pd.DataFrame(all_rows, columns=['place_id','lat','lon','business_status','name','vicinity'])
        api_point_df.to_csv(f"google_point_{point_id}_page_{j}.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [64]:
# Read every station returned by api calls
all_gas_stations = []
path = '/Users/zack/Desktop/stl/wsdot_evse/google_api_calls/csv/'
for filename in os.listdir(path):
    data = pd.read_csv(path+filename)
    all_gas_stations.append(data)
all_gas_stations = pd.concat(all_gas_stations)

# Remove temporarily closed stations, and duplicates
all_gas_stations.drop(columns='Unnamed: 0', inplace=True)
all_gas_stations = all_gas_stations[all_gas_stations['business_status']=="OPERATIONAL"]
all_gas_stations.drop_duplicates(inplace=True)
all_gas_stations.to_csv(path+"../../all_gas_stations.csv")

In [65]:
all_gas_stations

,place_id,lat,lon,business_status,name,vicinity
0,ChIJKw5qc63FglQRR5RCtENkfXE,49.036946,-119.478585,OPERATIONAL,Husky,"Rr2 Site 78 Comp 23 Hwy 97 &, British Columbia..."
1,ChIJseOnnVLEglQRI-cshQzS8IU,49.036779,-119.477378,OPERATIONAL,Shell,"9121 92 Avenue, Osoyoos"
2,ChIJg_zZrTjqglQRHnjIeo22oOE,49.159911,-119.553006,OPERATIONAL,Moresby Fuels Ltd,"91st Street, Oliver"
3,ChIJmX8GmLLFglQRgg-pBd5ithc,49.034517,-119.474336,OPERATIONAL,7-Eleven,"9131 Main Street, Osoyoos"
4,ChIJVTxDPrLFglQR5MhYXMRI700,49.032696,-119.471199,OPERATIONAL,Osoyoos Mohawk,"9101 Main Street, Osoyoos"
...,...,...,...,...,...,...
15,ChIJz5nNPJU6kFQR0foXuHLVeaA,47.652544,-122.690458,OPERATIONAL,Safeway Fuel Station,"2906 Northwest Bucklin Hill Road, Silverdale"
16,ChIJ0zUVkd4kkFQRPNCcv9yURDQ,47.693672,-122.692560,OPERATIONAL,76,"14367 Clear Creek Road Northwest, Silverdale"
17,ChIJxeNmjMM6kFQRH9BFVhKYHO8,47.661011,-122.684270,OPERATIONAL,76,"10744 Silverdale Way Northwest, Silverdale"
18,ChIJBURDB8I6kFQRN4NeRUA-ux0,47.655811,-122.679297,OPERATIONAL,Costco Gas Station,"10000 Mickelberry Road Northwest, Silverdale"
